In [11]:
# Import dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

ModuleNotFoundError: No module named 'citipy'

In [3]:
# Create a new set of 2,000 random latitudes and longitudes
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Unpack lat_lngs zip object into a list. This way, we only need to create a set of random latitudes and longitudes once. 
lat_lngs = zip(lats, lngs)
lat_lngs

# One can only unzip a zipped tuple once before it is removed from the computer's memory. 
# Make sure to unzip the latitudes and longitudes into the coordinates list before moving on.

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print (coordinates)

[(87.43567355017439, -130.42084619166235), (-56.42692918357154, 150.6551401074056), (-38.83687369275977, -18.565623514206237), (25.73639414754568, -137.81539667452), (-48.369220934252, 126.58593084425013), (13.82043944321039, 173.9301894566476), (-24.374113876277193, -31.059244644998586), (-65.64039162402575, -159.11788710696834), (81.21066489888753, -166.28498329116812), (55.84868001132304, 73.93069013049848), (-75.54121437269932, -62.43762160024089), (-61.911554990284706, -140.26756302787294), (-42.01773737570418, 102.591467966376), (-54.79217150374996, -69.3945790637835), (-37.33814710909199, 84.01090720093947), (-85.51816947624027, 99.85460348809818), (-5.859348251180009, -21.499735915906683), (48.701969261459055, 26.089103022373877), (85.86934546295683, 168.23492690606662), (-23.066792935954254, 167.1208078665294), (51.38865487931315, -175.95837849717637), (-33.95004527944688, 67.57015373870556), (83.61072357202374, 31.694351531013893), (54.19999252864662, 12.599016270699991), (-6

In [5]:
# Get the nearest city using the citipy module
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

NameError: name 'citipy' is not defined

In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

NameError: name 'weather_api_key' is not defined

In [7]:
# Retrieve the following information from the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
# Tally the number of cities in the city_data array of dictionaries
len(city_data)

0

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

""


In [ ]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")